# MATHS1004 Mathematics for Data Science I
## Computer Lab 5

This lab is partly about text generation using Markov chains, but also (really) an illustration of *data wrangling* -- one of the essential aspects of practical data science. Actual data scientists say that wrangling data into a useable form is [80% of the job](https://www.nytimes.com/2014/08/18/technology/for-big-data-scientists-hurdle-to-insights-is-janitor-work.html)! 

Along the way we'll learn about:
- reading data into Python from a URL;
- dealing with strings;
- dictionaries! (Possibly **the best** aspect of using Python);
- problem-solving with messy data.

Here we go. 

### Data wrangling

In this section we need to read a piece of text in, clean it, then create a list of words from it. We'll need the following libraries, so load them up.

In [ ]:
import numpy as np
import urllib.request, string
from collections import Counter

`urrllib.request` allows you to access data directly from a URL. The following block loads a text file from the URL given, and then reads the first line. Run it to find out the dataset we'll be working with.

In [ ]:
URL = 'http://www.site.uottawa.ca/~lucia/courses/2131-02/A2/trythemsource.txt'

data = urllib.request.urlopen(URL)
data.readline()

Fun!!! Part of the reason this book is famous is because it [uses only 50 different words](https://en.wikipedia.org/wiki/Green_Eggs_and_Ham), making it perfect for exploring text generation.

The first thing you notice here is that we have a weird `b` before the string prints. A little bit of Googling will tell you that this means the string is encoded in a format called "UTF-8". This will cause us pain later on! So the first step will be to decode it to a regular ASCII string. Copy the last two commands above into the cell below, then put a `.decode('utf-8')` at the end of the `data.readline()` command, then execute.

In [ ]:
data = urllib.request.urlopen(URL)
data.readline().decode('utf-8')


The `b` should be gone -- great! We now have an ASCII string.

Now, let's look at the whole document, line by line. Loop over the entire object `data`, and print each line. I'll start you off, then you fill in the rest of the loop.

In [ ]:
for line in data:


To make the list of words we need for this project the best thing to do is to first make this whole document into one long string. So we need to append each line into one string. Fortunately, you can do this in Python easily by adding strings. For example:

In [ ]:
text = "Do you like green eggs and Ham?"
line = "I do not like them, Sam-I-Am."

text = text + line
print(text)

(Question: How could you put in a space between the first and second sentences without changing them?)

Try using this idea to build your own variable `text` containing the entire document. We'll need to reload from the `URL`, so again I'll start you off. You fill in the rest of the `for` loop.

In [ ]:
data = urllib.request.urlopen(URL)
data.readline().decode('utf-8')

text = ''
for line in data:

    
    
text

If you got an error: did you decode each line to an ASCII string? Make sure you use the `decode` function.

Once you've got the long `text`, you'll notice another issue -- we have `\r`'s and `\n`'s all over the place! These are newline characters, which will put gaps in if you `print(text)` (try it). But they're a problem! You'll need to strip them out. Fortunately there's another helpful function: appending `.rstrip()` will strip out special characters from the right of a string, leaving us with only the text. Put this in to your loop above to end up with just text.


Once `text` contains only text, we move on to the next issue: to do text generation we need to have only words, no punctuation! So we should replace every piece of punctuation with a space. You can Google how to do this, or I can do it for you and find the code snippet I found...

In [ ]:
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
text = text.translate(translator)
text

Looking pretty close. But can you see an issue? Look on the 6th line from the bottom, do you see some words in lower case? Whoever made this text file has added something in there, a description of a picture from the book. How annoying for us! Those words aren't part of the original story, and so we should remove them. Look up the [replace](https://www.geeksforgeeks.org/python-string-replace/) command, and use it to remove that piece of text. Then, let's convert everything to lower-case, by appending `.lower()` to the end of `text`.

Now. How many unique words are there in `text`? The code below splits `text` (by spaces) into a list of words. Then I create a set from that list (i.e., all the unique elements), and take the length. This gives the vocabulary size, or the number of unique words.

In [ ]:
words = text.split()
len(set(words))

I get 52 different words. What??? Where did those extra 2 words come from? Something is messy with the dataset itself, and we have to do some legit detective work to track the issue down. Here's how I figured it out. `Counter` creates a dictionary of how many times each item appears in a list. And `.most_common()` sorts that list from most to least common. 

Run the cell below, then look towards the bottom. I won't spoil your fun -- go figure out what the origin of those two extra words is. (Hint: You might need to visit the original URL and do some searching/reading...)

In [ ]:
Counter(words).most_common()

Once you've figured it out, go ahead and `.replace` those two words with what they ought to be! Then check that the vocabulary size is in fact 50 words.

For the purposes of this lab, we can now consider this dataset *cleaned*, and we can proceed to doing some text generation using the list `words`. Think about the steps involved with the cleaning, the puzzles to solve, and the care we had to take. This is the essence of doing data science in practice.

Let's now move on to the original aim of the lab!

### Text generation using Markov chains

Text generation is important in modern tech -- it forms the basis for predictive text on your smartphone, as well as [topic modelling](http://www.cs.columbia.edu/~blei/papers/Blei2012.pdf), image captioning, and text summarisation. Recently a computational model called GPT-2 was deemed "[too dangerous to release](https://www.theguardian.com/technology/2019/feb/14/elon-musk-backed-ai-writes-convincing-news-fiction)" by its own creators!

We will take no such precautions here.

One foundation of text generation is Markov chains, a stochastic model where the probability of being in a particular state at time $k+1$ depends on where you were at time $k$:
$$
{\bf x}_{k+1} = P{\bf x}_k
$$
In a Markov chain, the vectors ${\bf x}_k$ are called *state* vectors and have the property that each
entry lies between 0 and 1 and their sum is one.  Vectors with this property
are called *probability* vectors.  The matrix $P$ is a square matrix
all of whose columns are probability vectors; such a matrix $P$ is called a
*stochastic* matrix.

We've already seen an example of a Markov chain: the *PageRank* example we did in lectures (from [here](https://www.rose-hulman.edu/~bryan/googleFinalVersionFixed.pdf)) was a Markov chain. Here's another one. 

Let ${\bf x}_0=(0.2,0.8)$ represent the
proportions of a population in the city and suburbs at time $k=0$.  The
transition from one year to the next can be given as a $2\times 2$ matrix:

$$
    P = \begin{array}{c@{}l}
        \text{From:} & \text{To:} \\
        \text{City Suburbs} & \\
        \begin{bmatrix} 
         \ 0.97\ &\ 0.05\ \\
         0.03 & 0.95\end{bmatrix} 
         & \begin{array}{l} \text{City}\\ 
         \text{Suburbs}\end{array} 
         \end{array}
$$
For $k=0,1,2,3,\dots$ we have ${\bf x}_{k+1}=P{\bf x}_k$.

Try using `numpy` and a `for` loop to calculate the proportions of population in city versus suburbs for a few years. What do the proportions converge to over time? (If you're interested in the theory behind this, PageRank, and Markov chains more generally, come back for the course MATHS 2103/7103 *Probability & Statistics*!)

How does this relate to text generation? Well, we can use our list `words` above to build a transition matrix $P$, make sure that the columns sum to 1 (think about how you might do this), and then simulate new text.

Of course, it's not very efficient to build a $50\times50$ transition matrix, and it doesn't scale well to larger vocabularies. A much more elegant approach is to build a *dictionary* of all the words appearing after each word in our text, and then sample from the lists in this dictionary. I adapted the following code to build this dictionary from [this nice tutorial](https://towardsdatascience.com/simulating-text-with-markov-chains-in-python-1a27e6d13fc6).

In [ ]:
def make_markov_chain_model(words):
    word_dict = {}
    for i in range(len(words)-1):
        word_1 = words[i]
        word_2 = words[i+1]
        if word_1 in word_dict.keys():
            word_dict[word_1].append(word_2)
        else:
            word_dict[word_1] = [word_2]
            
    return(word_dict)

Use the function above to build a dictionary from the `words` list you built above. Take a look at the result.

To simulate text, we start with a word then sample from the list of words following it in our dictionary to create new text. Here's a function to do this:

In [ ]:
def simulate_text(n_words,first_word,word_dict):
    chain = [first_word]
    for i in range(n_words):
        chain.append(np.random.choice(word_dict[chain[-1]]))

    simulated_text = ' '.join(chain)
    return(simulated_text)

Choose a starting word an use this function to simulate, say, 50 words of original "Green Eggs and Ham" text. How does it read? 

Some questions for you to explore:
- What happens if you use different starting words? For example, try starting with "if".
- What happens if you use a word that wasn't in the original text?
- How might you incorporate sentence structure, rather than just arbitrary lengths of text?
- How might you make the generated text more realistic? (You might like to look up what a "higher-order" Markov chain is.) Can you think of any computational issues that might arise here?
- What about generating text in the style of different authors? You might want to look into "training" a Markov chain model on works from [Project Gutenberg](https://www.gutenberg.org), which can be accessed [directly](https://pypi.org/project/Gutenberg/) from Python.

There's also a nice package called [markovify](https://github.com/jsvine/markovify) which makes building the Markov chains earier. Check it out!

Finally, notice how Markov chains blend two different topics together: linear algebra and probability. This is another characteristic of practical data science -- it uses a broad range of mathematical skills!